# Vehicle control in SymuVia
## API basic controls

In order to control vehicles in SymuVia it is required the library `libSymuVia.dylib` with exported functions: 

* The library is able to perform simulations in three ways 
    * ***Case 1:*** Launch simulation: equialent to the graphical interface (just via python) 
    * ***Case 2:*** Launch step by step

## Step 1: Import the SymuVia Library

Import the modules to make visible the library from python 

In [1]:
# import tasks 
from ctypes import cdll
import os

Determine the location of the file `libSymuVia.dylib` in MacOS or `SymuVia.dll` in Windows

In [2]:
# library path
path_to_library = 'darwin'
lib_name = 'libSymuVia.dylib'
full_path = os.path.join(os.getcwd(),path_to_library,lib_name)
if os.path.isfile(full_path):
    print(full_path)
# !ls $full_path

/Users/ladino/Documents/03-Code/02-Python/practices/demo-symuvia/darwin/libSymuVia.dylib


Charge the library within Python 

In [3]:
# simulator loading
symuvia = cdll.LoadLibrary(full_path)
# symuvia 

## Step 2: Find the simulation File 

Try with different filenames, here there to examples: `1_scenario_hdv_90_chdv_10.xml` or `bottleneck_001.xml`, first find the exact localization in the folder:

In [4]:
# simulation path 
simulation_filename = '1_scenario_hdv_90_chdv_10.xml'
# simulation_filename = 'bottleneck_001.xml'
full_path_file = os.path.join(os.getcwd(),simulation_filename)
print(full_path_file)
# !ls $sim_file

/Users/ladino/Documents/03-Code/02-Python/practices/demo-symuvia/1_scenario_hdv_90_chdv_10.xml


Make visible the simulation file to SymuVia

In [5]:
# load simulation into symuvia 
m = symuvia.SymLoadNetworkEx(full_path_file.encode('UTF8'))


## Step 3: Launch a Simulation 

The command `SymRunEx` launches a single run

In [6]:
# launch a simulation 
symuvia.SymRunEx(full_path_file.encode('UTF8'))
# returning 0 means success, find all data in : Sim_output_90_10 or test_output

0


## Step 4: Launch a Simulation (Step by Step)

To create a simulation by steps it is required to create a buffer, and some pointers towards c. For the following steps it is also necessary to install the package `xmltodict` which parses the XML output from SymuVia into a Python dictionary. More info about installation of this package here [link](https://pypi.org/project/xmltodict/). Install via pip `pip install xmltodict` or via conda (recommended) `conda install -c conda-forge xmltodict`. 

***Note***: In case it is not clear how to install packages on anaconda follow this link [here](https://www.youtube.com/watch?v=Z_Kxg-EYvxM)

In [7]:
from ctypes import create_string_buffer, c_int, byref, c_bool, c_double
from xmltodict import parse

The following code initializes the pointers 

In [8]:
# pointers 
sRequest = create_string_buffer(100000) # Size of buffer -> Depends on number of current vehicles at time t 
bEnd = c_int() #
bTrace = c_bool(False) # Do we want to store xml traces?

This code will launch the simulation step by step. First `N_steps` define the number of steps that will be launched. Each time the instruction `symuvia.SymRunNextStepEx` is launched a new step is simulated. At each time step all data is within an `OrderedDict` (a dictionary), in the following example everything is contained in `data`

In [9]:
# start simulation from beginning
m = symuvia.SymLoadNetworkEx(full_path_file.encode('UTF8'))

N_steps = 8 # run 3 iterations 
step = iter(range(N_steps))

bContinue = True 
while bContinue:
    try:
        t = next(step)
        bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
        data = parse(sRequest.value.decode('UTF8'))
        ti = data['INST']['@val']
#         print(sRequest.value.decode('UTF8'))
        print(f"t: {ti}", bEnd)
        print(f"it: {t}", data['INST']['TRAJS'])
        bContinue = not bEnd
    except StopIteration:
        print(f"Iteration: {t}, time: {ti}")
        bContinue = False 
    

t: 0.10 c_int(0)
it: 0 None
t: 0.20 c_int(0)
it: 1 None
t: 0.30 c_int(0)
it: 2 None
t: 0.40 c_int(0)
it: 3 None
t: 0.50 c_int(0)
it: 4 None
t: 0.60 c_int(0)
it: 5 None
t: 0.70 c_int(0)
it: 6 None
t: 0.80 c_int(0)
it: 7 OrderedDict([('TRAJ', OrderedDict([('@abs', '-999.38'), ('@acc', '0.00'), ('@dst', '0.62'), ('@id', '0'), ('@ord', '148.50'), ('@tron', 'Zone_A'), ('@type', 'CHDV'), ('@vit', '20.00'), ('@voie', '1'), ('@z', '0.00')]))])
Iteration: 7, time: 0.80


## Step 5: Vehicle creation

For vehicle creation it is important to define: 
 * Vehicle type `str`
 * Origin `str`
 * Destination `str`
 * Lane Number `int`
 * Time instant `float` 
 
 **Important**: *Time instant* = first time step of the simulation, all vehicles should be first *created* at first time step, then *driven* at all following time steps.  Vehicle that is not followed for first time can be driven after. This behavior can be then modified. 

 
 **Procedure to drive**:
 1. Create the vehicle 
 2. Log/register the vehicle within the network `SymRunNextStepEx`
 3. Drive the vehicle `SymDriveVehicleEx` 
 4. To make effective just push the new posiiton via `SymRunNextStepEx`

    

In [10]:
# start simulation from beginning
m = symuvia.SymLoadNetworkEx(full_path_file.encode('UTF8'))


# Run til @ 0.1 
N_steps = 1
step = iter(range(N_steps))

bContinue = True 
while bContinue:
    try:
        t = next(step)
        bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
        data = parse(sRequest.value.decode('UTF8'))
        ti = data['INST']['@val']
#         print(sRequest.value.decode('UTF8'))
        print(f"it: {t}", data['INST']['TRAJS'])
        bContinue = not bEnd
    except StopIteration:
        print(f"Iteration: {t}, time: {ti}")
        bContinue = False

# @ t+1
# vehicle declaration 
stype = 'CHDV'
sOrigin= 'Ext_In'
sDestination = 'Ext_Out'
nVoie = c_int(1)
dbTime = c_double(0.1)
nIdVeh = symuvia.SymCreateVehicleEx(stype.encode('UTF8'),
                                    sOrigin.encode('UTF8'),
                                    sDestination.encode('UTF8'),
                                    nVoie,
                                    dbTime)
print(f'\nVehicle id {nIdVeh} created:' )
bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
data = parse(sRequest.value.decode('UTF8'))
ti = data['INST']['@val']
print(f'Vehicle has been logged at time {ti}')

# Vehicle won't be printed at this time step
print(f"\ntime: {ti}")
print(data['INST']['TRAJS'])

# @ t+1 
# (optional - better to avoid)
# By doing this, the initial position is initialized by the simulator
# Vehicle will be printed at this time step 
# bSuccess =  symuvia.SymRunNextStepEx(sRequest, True, byref(bEnd))
# data = parse(sRequest.value.decode('UTF8'))
# ti = data['INST']['@val']

# print(f"\ntime: {ti}")
# print(data['INST']['TRAJS'])


# @ t+1
# Vehicle will be driven to a specific position (forcing position) 
# It can be done after simulation 
dPos = c_double(2.5)
sLink = 'Zone_A'
nres = symuvia.SymDriveVehicleEx(nIdVeh,
                                 sLink.encode('UTF8'), 
                                 nVoie, 
                                 dPos, 
                                 1)

bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
data = parse(sRequest.value.decode('UTF8'))
ti = data['INST']['@val']
print(f'\nVehicle position has been updated at time {ti}')
print(f"time: {ti}")
print(data['INST']['TRAJS'])

it: 0 None
Iteration: 0, time: 0.10

Vehicle id 0 created:
Vehicle has been logged at time 0.20

time: 0.20
None

Vehicle position has been updated at time 0.30
time: 0.30
OrderedDict([('TRAJ', OrderedDict([('@abs', '-997.50'), ('@acc', '50.00'), ('@dst', '2.50'), ('@etat_pilotage', 'force (ecoulement non respecte)'), ('@id', '0'), ('@ord', '148.50'), ('@tron', 'Zone_A'), ('@type', 'CHDV'), ('@vit', '25.00'), ('@voie', '1'), ('@z', '0.00')]))])


Once created the vehicle can be handled by the simulator as expected 

It is possible also to drive partially a vehicle 

In [11]:
bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
data = parse(sRequest.value.decode('UTF8'))
ti = data['INST']['@val']
print(f'Vehicle position has been updated at time {ti}')
print("\ntime: {}".format(ti))
print(data['INST']['TRAJS'])

Vehicle position has been updated at time 0.40

time: 0.40
OrderedDict([('TRAJ', OrderedDict([('@abs', '-995.50'), ('@acc', '-50.00'), ('@dst', '4.50'), ('@id', '0'), ('@ord', '148.50'), ('@tron', 'Zone_A'), ('@type', 'CHDV'), ('@vit', '20.00'), ('@voie', '1'), ('@z', '0.00')]))])


## Step 6: Drive an existing vehicle 

To drive the vehicle it is required just to identify the `id` and provide informaiton about `position`, `lane`, `link`

In [12]:
# start simulation from beginning
m = symuvia.SymLoadNetworkEx(full_path_file.encode('UTF8'))

# Run til @ 0.1 
N_steps = 8
step = iter(range(N_steps))

bContinue = True 
while bContinue:
    try:
        t = next(step)
        bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
        data = parse(sRequest.value.decode('UTF8'))
        ti = data['INST']['@val']
#         print(sRequest.value.decode('UTF8'))
        print(f"it: {t}", data['INST']['TRAJS'])
        bContinue = not bEnd
    except StopIteration:
        print(f"Iteration: {t}, time: {ti}")
        bContinue = False


print("\ntime: {}".format(data['INST']['@val']))
print(data['INST']['TRAJS'])       

nIdVeh = 0
nVoie = c_int(1)
dPos = c_double(1)
sLink = 'Zone_A'
nres = symuvia.SymDriveVehicleEx(nIdVeh,sLink.encode('UTF8'), nVoie, dPos, 1)
bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
data = parse(sRequest.value.decode('UTF8'))
ti = data['INST']['@val']
print(f'\nVehicle position has been updated at time {ti}')
print("time: {}".format(ti))
print(data['INST']['TRAJS'])

it: 0 None
it: 1 None
it: 2 None
it: 3 None
it: 4 None
it: 5 None
it: 6 None
it: 7 OrderedDict([('TRAJ', OrderedDict([('@abs', '-999.38'), ('@acc', '0.00'), ('@dst', '0.62'), ('@id', '0'), ('@ord', '148.50'), ('@tron', 'Zone_A'), ('@type', 'CHDV'), ('@vit', '20.00'), ('@voie', '1'), ('@z', '0.00')]))])
Iteration: 7, time: 0.80

time: 0.80
OrderedDict([('TRAJ', OrderedDict([('@abs', '-999.38'), ('@acc', '0.00'), ('@dst', '0.62'), ('@id', '0'), ('@ord', '148.50'), ('@tron', 'Zone_A'), ('@type', 'CHDV'), ('@vit', '20.00'), ('@voie', '1'), ('@z', '0.00')]))])

Vehicle position has been updated at time 0.90
time: 0.90
OrderedDict([('TRAJ', OrderedDict([('@abs', '-999.00'), ('@acc', '-161.54'), ('@dst', '1.00'), ('@etat_pilotage', 'force (ecoulement respecte)'), ('@id', '0'), ('@ord', '148.50'), ('@tron', 'Zone_A'), ('@type', 'CHDV'), ('@vit', '3.85'), ('@voie', '1'), ('@z', '0.00')]))])


what happens with vehicles that are not driven anymore?

**R/:**  They continue their evolution as if they were under car-following.

In [13]:
bSuccess =  symuvia.SymRunNextStepEx(sRequest, bTrace, byref(bEnd))
data = parse(sRequest.value.decode('UTF8'))
ti = data['INST']['@val']
print(f'Vehicle position has been updated at time {ti}')
print("\ntime: {}".format(ti))
print(data['INST']['TRAJS'])

Vehicle position has been updated at time 1.00

time: 1.00
OrderedDict([('TRAJ', OrderedDict([('@abs', '-998.60'), ('@acc', '1.50'), ('@dst', '1.40'), ('@id', '0'), ('@ord', '148.50'), ('@tron', 'Zone_A'), ('@type', 'CHDV'), ('@vit', '4.00'), ('@voie', '1'), ('@z', '0.00')]))])


Author: Andres Ladino